In [2]:
import os
from itertools import product
import re

import django
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.cm import register_cmap
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA, FactorAnalysis
import marslab.spectops as ops
from fit import correlation_matrix
from marslab.imgops.imgutils import normalize_range
from marslab.compat.xcam import DERIVED_CAM_DICT
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
from marslab.imgops.pltutils import attach_axis, set_colorbar_font


from correlate import (
    s_from_midnight, explode_binary, translate_fields_for_corr_graphs,
    preprocess_for_corrs, plot_dimensionality_matrices,
    plot_mdex_pca
)

# os.chdir("..")

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()
import plotter.models
from plotter.spectrum_ops import filter_df_from_queryset
from multidex_utils import model_metadata_df

%matplotlib qt
%matplotlib qt

In [3]:
def explained_variance_ratios(array):
    variances = array.var(axis=0)
    total = sum(variances)
    return variances / total

In [5]:
instrument = "ZCAM"
filter_info = DERIVED_CAM_DICT[instrument]["filters"]
filters = list(filter_info.keys())
narrowband = [filt for filt in filters if not re.match(r"[LR]0[RGB]", filt)]


In [ ]:
plt.rcParams["figure.figsize"] = (11, 11)
corrs = []
transforms = []
figlist = []
for r_star, scale_to, search_terms, norm_values, filts in product(
    [False, True],
#     [None, ("L1", "R1")],
    [None],
    [None],
#     [None, [("feature", "rock")]],
    [False, 'R6'],
#     [PCA, FactorAnalysis]
    [filters, narrowband]
    
):
#     if search_terms is not None:
#         explode_field = "morphology"
#     else:
#         explode_field = None
    if (scale_to is not False) and (r_star is True):
        continue     # pointless, move on
    explode_field=None
    method=PCA
    correlations, transform, figs = plot_mdex_pca(
        scale_to = scale_to,
        instrument=instrument,
        explode_field = explode_field,
        r_star = r_star,
        search_terms = search_terms,
        corr_fields = filts + [
            'incidence_angle'
        ],
        pca_fields=filts,
        corr_cmap="orange_teal",
        fontsize=18,
        method=method,
        norm_values=norm_values
        
    )
    corrs.append(correlations)
    transforms.append(transform)
    figlist.append(figs)

In [ ]:
plt.close('all')

In [ ]:
for figs in figlist:
    figs['pc correlations'].show()

In [24]:
from fit import correlation_matrix

In [34]:
correlation_matrix(corpus[['zoom', 'sol']])

,zoom,sol
zoom,1.00,0.51
sol,0.51,1.00


In [32]:
corpus['zoom']

1      110
2      110
3      110
4      110
5      110
      ... 
317     34
318     63
319     63
320     63
321     63
Name: zoom, Length: 321, dtype: object

In [50]:
instrument='ZCAM'
spec_model = plotter.models.INSTRUMENT_MODEL_MAPPING[instrument]
wave_mapping = pd.Series(DERIVED_CAM_DICT[instrument]['filters'])
narrowband = [filt for filt in filters if not re.match(r"[LR]0[RGB]", filt)]
filter_info = DERIVED_CAM_DICT[instrument]["filters"]
filters = list(filter_info.keys())
metadata_df = model_metadata_df(spec_model)
search_terms=None
norm_values = False
pca_fields = filters
# corr_fields = filters + ["incidence_angle", 'r16', 'r56', 'l645']
# corr_fields = filters + ["incidence_angle", "zoom", 'sol']
corr_fields = filters + ["incidence_angle", 'ltst', 'sclk']

explode_field = None
r_star=False
# scale_to=('L1', 'R1')
scale_to=None
# scale_to=('L6', 'R6')

data_df = filter_df_from_queryset(
    spec_model.objects.all(), r_star=r_star, scale_to=scale_to
)
corpus = pd.concat([metadata_df, data_df], axis=1)
corpus["ltst"] = corpus["ltst"].map(s_from_midnight)
corpus["avg"] = corpus[filters].mean(axis=1)
corpus['r56'] = corpus['R5'] / corpus['R6']
corpus['r16'] = corpus['R1'] / corpus['R6']
corpus['l645'] = ops.band_depth(
    corpus[['L6', 'L4', 'L5']].T,
    None,
    wave_mapping[['L6', 'L4', 'L5']]
)[0]
if 'zoom' in corpus.columns:
    corpus['zoom'] = corpus['zoom'].astype('float16')
if (explode_field is not None) and (explode_field in corpus.columns):
    exploded = explode_binary(corpus, explode_field)
    search = pd.concat([corpus.copy(), exploded], axis=1)
else:
    exploded = None
    search = corpus.copy()

# fields to do pca on
pca_fields = translate_fields_for_corr_graphs(filters, pca_fields)

# fields to compare with the PCs
corr_fields = translate_fields_for_corr_graphs(filters, corr_fields)
if exploded is not None:
    corr_fields += list(exploded.columns)
# corr_fields += [band + "_err" for band in filters]
pca_data, corr_data = preprocess_for_corrs(
    corr_fields, norm_values, pca_fields, search, search_terms
)

In [51]:
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.pipeline import Pipeline
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('reduce', PCA(n_components=8))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]
if "explained_variance_ratio_" in dir(pipe.named_steps['reduce']):
    explained_variance = np.round(
        pipe.named_steps['reduce'].explained_variance_ratio_ * 100, 2
    )
else:
    explained_variance = None
corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    explained_variance=explained_variance,
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

{'parameters': <Figure size 640x480 with 2 Axes>}

In [107]:
from sklearn.decomposition import FastICA
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('ica', FastICA(n_components=8, whiten=True))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]

corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

{'parameters': <Figure size 640x480 with 2 Axes>}

In [120]:
from sklearn.decomposition import NMF
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
#     ('scale', StandardScaler()),
    ('nmf', NMF(n_components=16, init='random', max_iter=5000))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]

corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

{'parameters': <Figure size 640x480 with 2 Axes>}

In [117]:
explained_variance_ratios(transform)

P1     0.194671
P2     0.227756
P3     0.028433
P4     0.037464
P5     0.032736
P6     0.035942
P7     0.019016
P8     0.019965
P9     0.025239
P10    0.023616
P11    0.051665
P12    0.025683
P13    0.086045
P14    0.001573
P15    0.036302
P16    0.153895
dtype: float64

In [103]:
from sklearn.decomposition import KernelPCA
vectors = pca_data.T.to_dict("list")
vectarray = np.array(tuple(vectors.values()))
pipe = Pipeline(steps=[
    ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('kernel', KernelPCA(n_components=16))
])
transform = pipe.fit_transform(vectarray)
transform = pd.DataFrame(transform)
transform.columns = ["P" + str(column + 1) for column in transform.columns]

corr_frame = pd.concat([corr_data, transform], axis=1)
correlations = correlation_matrix(corr_frame)

plot_dimensionality_matrices(
    correlations, 
    transform, 
    corr_fields, 
    corr_cmap='orange_teal', 
    fontsize=20,
    which='parameters',
    title=" ".join(pipe.named_steps.keys())

)

{'parameters': <Figure size 640x480 with 2 Axes>}

In [99]:
np.array([1,1,2,3,3]).var(axis=0)

0.8

P1     0.672857
P2     0.190888
P3     0.044724
P4     0.035118
P5     0.016631
P6     0.012657
P7     0.007040
P8     0.006082
P9     0.003611
P10    0.002769
P11    0.002035
P12    0.001861
P13    0.001277
P14    0.001068
P15    0.000794
P16    0.000587
dtype: float64

In [66]:
from sklearn.metrics import explained_variance_score

In [71]:
transform

,P1,P2,P3,P4,P5,P6,P7,P8
0,-2.267093,-1.724095,0.434542,-0.182625,-0.187267,-0.933902,-1.156069,0.546855
1,-2.526464,1.224259,0.746468,0.138954,0.066854,-0.306094,0.651362,-0.030773
2,-5.165960,-0.126977,-0.271500,0.345461,0.035312,-0.422036,0.446803,-0.135420
3,-1.344621,1.834564,0.297798,0.390662,0.517248,-0.801794,0.171038,0.062085
4,-3.907494,-0.984149,1.127406,0.517246,0.036090,-0.646001,0.476109,-0.586003
...,...,...,...,...,...,...,...,...
308,0.505675,-0.630974,-0.267743,0.622051,0.131681,0.234267,-0.224845,-0.177420
309,-0.012375,-0.093916,-0.510644,0.184119,-0.180987,0.051594,-0.162309,-0.410684
310,-1.503885,-0.517673,-0.687886,-0.109053,-0.038638,0.236416,0.069289,-0.567178
311,-5.692225,0.172917,-0.866286,0.416134,0.062320,0.049577,-0.089146,-0.441941


In [73]:
vectarray.shape

(313, 18)

In [77]:
from fit import coef_det

In [ ]:
plot_dimensionality_matrices?